# [NML-25] Notebook 2: Spectral Graph Theory

Responsible TA: [William Cappelletti](https://people.epfl.ch/william.cappelletti)

## Instructions


**Expected output:**

Troughout the different lab session, you will have coding and theoretical questions. Coding exercises shall be solved within the specified space:
```python
# Your solution here ###########################################################
...
#^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
```
Sometimes we provide variable names, such as `x = ...`; do not change names and stick to hinted typing, as they will be reused later.
Within the solution space, you can declare any other variable of function that you might need, but anything outside these lines shall not be changed, or it will invalidate your answers.

Theoretical questions shall be answered in the following markdown cell. The first line will be 
```markdown
**Your answer here:**
...
```

**Solutions:**
* Your code should be self-contained in the `.ipynb` file. The solution to the exercices will be provided in an external `.ipynb` file.

* Try to make your code clean and readable, it is a good training for the project. Provide meaningful variable names and comment where needed.

* You cannot import any other library than we imported, unless explicitly stated.

## Objective

This assignment features three exercises: 
  - spectral clustering
  - spectral graph filtering


NB: You are encouraged to try different random seeds, remember to re-run the whole notebook after doing so.

### Setup

In [ ]:
import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
from sklearn.cluster import KMeans

rng = np.random.default_rng(235486)

## Section I: Spectral clustering

In this exercise, we will perform both positional-feature-based as well as spectral clustering on two different graph structures (disjoint box graphs and crescent moon) and analyze the influence of the features and graph on the obtained clusters.

*For additional information on spectral clustering, we refer to this [tutorial paper](http://www.tml.cs.uni-tuebingen.de/team/luxburg/publications/Luxburg07_tutorial.pdf). For practical aspects of the algorithm, the relevant sections are 1, 2, 3, 4, 8.*

### Load and visualize data

In [ ]:
def get_boxes(num_boxes: int = 5, num_random_edges: int = 5):
    boxes = [nx.grid_2d_graph(m=n, n=n) for n in range(3, 3 + num_boxes)]
    pos = np.concatenate(
        [
            np.array(list(nx.kamada_kawai_layout(box).values())) + i * 1.25
            for i, box in enumerate(boxes)
        ]
    )
    G = nx.disjoint_union_all(boxes)
    return G, pos

In [ ]:
def get_moons(num_moons: int = 5, num_points: int = 1000, eps: float = 2.5e-2):
    points = rng.uniform(-1, 1, size=(num_points, 2))
    x, y = points[:, 0], points[:, 1]
    circ = x**2 + y**2 <= 1
    circ2 = (x - 0.5) ** 2 + y**2 <= 1
    moon = points[circ & ~circ2]
    moons = np.concatenate(
        [
            np.array([1 - 2 * (i % 2), 1]) * (moon + np.array([0, i * 1.5]))
            for i in range(num_moons)
        ]
    )
    moon_dists = np.sum((moons[np.newaxis] - moons[:, np.newaxis]) ** 2, axis=2)
    moon_edges = np.vstack(((0 < moon_dists) & (moon_dists < eps)).nonzero()).T
    moon_graph = nx.empty_graph(n=len(moons))
    moon_graph.add_edges_from(moon_edges)
    return moon_graph, moons

In [ ]:
def plot_graph(G, pos, node_clusters=None):
    plt.figure(figsize=(5, 5))
    nx.draw(
        G,
        pos=pos,
        node_size=10,
        node_color=(
            [plt.cm.Set1(c) for c in node_clusters]
            if node_clusters is not None
            else "gray"
        ),
    )
    plt.show()

In [ ]:
num_clusters = 5
G_boxes, G_boxes_pos = get_boxes(num_clusters)
plot_graph(G_boxes, G_boxes_pos)

In [ ]:
G_moons, G_moons_pos = get_moons(num_clusters)
plot_graph(G_moons, G_moons_pos)

### Part 1: K-means

#### Question 1.1: Run K-means with 5 clusters on both graphs.

You can use the scikit learn function (https://scikit-learn.org/stable/modules/generated/sklearn.cluster.KMeans.html).

In [ ]:
# Your solution here ###########################################################

G_boxes_clusters = ...
G_moons_clusters = ...
# ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^

In [ ]:
plot_graph(G_boxes, G_boxes_pos, G_boxes_clusters)
plot_graph(G_moons, G_moons_pos, G_moons_clusters)

#### Question 1.2: Explain the limitations of K-means with respect to these data.

**Your answer here:**

### Part 2: Spectral clustering

#### Question 2.1: Fill `compute_laplacian`, `compute_number_connected_components`, `spectral decomposition`.

These functions should work for all 3 definitions of the laplacian (combinatorial, symmetric normalized, random walk).

*Warning:* the eigendecomposition of a non symmetric matrix returns complex numbers, even if the imaginary part is in fact 0.

In [ ]:
def compute_laplacian(adjacency: np.ndarray, normalize: str):
    """normalize: can be None, 'sym' or 'rw' for the combinatorial, symmetric normalized or random walk Laplacians
    Return:
        L (n x n ndarray): combinatorial or symmetric normalized Laplacian.
    """
    # Your solution here ###########################################################
    ...

    if normalize is None:
        ...

    if normalize == "sym":
        ...

    elif normalize == "rw":
        ...
    # ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^

    else:
        raise ValueError(f"Unknown normalization: {normalize}")


def compute_number_connected_components(l_eigval: np.array, eps: float = 1e-12):
    """

    Args:
        l_eigval (np.array): eigenvalues of a Laplacian
        threshold (float, optional): cutoff value for very small numbers

    Returns:
        n_components (int): number of connected components.
    """
    # Your solution here ###########################################################
    ...
    # ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^


def spectral_decomposition(laplacian: np.ndarray):
    """Return:
    l_eigvec (np.array): eigenvalues of the Laplacian
    l_eigvals (np.ndarray): corresponding eigenvectors.
    """
    # Your solution here ###########################################################
    ...
    # ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^

#### Question 2.2: Compute how many connected components each of the graphs has.

Does the result change according to the laplacian definition?


In [ ]:
# Your solution here ###############################################################


# ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^

#### Question 2.3: Implement spectral clustering, then compute it and plot it for each Laplacian definition

Implement the `SpectralClustering` class, using the previously defined `compute_laplacian` and `spectral_decomposition`.

In [ ]:
class SpectralClustering:
    def __init__(self, n_classes: int, normalize: str):
        self.n_classes = n_classes
        self.normalize = normalize
        # Your solution here ###########################################################
        self.clustering_method = ...
        # ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^

    def fit_predict(self, G: nx.Graph):
        """Your code should be correct both for the combinatorial
        and the symmetric normalized spectral clustering.
        Return:
        y_pred (np.ndarray): cluster assignments.
        """
        # Your solution here ###########################################################
        ...
        # ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^

In [ ]:
# Combinatorial Laplacian
print("Spectral clustering with combinatorial Laplacian")

# Your solution here ###################################################################

...

# ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^

In [ ]:
# Combinatorial Laplacian
print("Spectral clustering with symmetric normalized Laplacian")

# Your solution here ###################################################################

...

# ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^

In [ ]:
# Combinatorial Laplacian
print("Spectral clustering with random walks Laplacian")

# Your solution here ###################################################################

...

# ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^

#### Question 2.4: Compare the spectral clustering obtained with the different Laplacians, and discuss about differences

**Your answer here:**

#### Question 2.5: Compare spectral clusters to K-means.

**Your answer here:**